<a href="https://colab.research.google.com/github/MaraElise/Analise_RegressaoLogistica/blob/main/KMV_Case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandasql


  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=288c40dec57c08267bdb192d883152ecfaa172b01f9185ca953bb7d216d7b3c2
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [ ]:
iimport pandas as pd
import pandasql as ps

In [ ]:
clientes = pd.read_csv('tb_clientes.csv', delimiter=',')
transacoes = pd.read_csv('tb_acumulo_pontos.csv', delimiter=',')

In [ ]:
clientes.head()

,IdCliente,Estado
0,3863356,RS
1,18158997,SP
2,36258332,SP
3,41800741,MA
4,27321774,SP


In [ ]:
transacoes.head()


,IdCliente,Pontos,ValorTransacao,AcumuloID,DataTransacao
0,42090967,0.0,0.0,1326791052,2024-03-01
1,39245562,-5.0,-5.0,1326503142,2024-03-01
2,18080820,7.0,6.7,1326823228,2024-03-01
3,41591184,7.0,7.0,1341036810,2024-03-01
4,28676128,10.0,10.0,1326848748,2024-03-01


In [ ]:
transacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53147 entries, 0 to 53146
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   IdCliente       53147 non-null  int64         
 1   Pontos          53147 non-null  float64       
 2   ValorTransacao  53147 non-null  float64       
 3   AcumuloID       53147 non-null  int64         
 4   DataTransacao   53147 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 2.0 MB


In [ ]:
# Converter a coluna 'data_str' para datetime
transacoes['DataTransacao'] = pd.to_datetime(transacoes['DataTransacao'])

In [ ]:
query_1 = """ select
              Estado,
              count(AcumuloID) as QtdTransacoes
              from transacoes
              left join clientes on transacoes.IdCliente = clientes.IdCliente
              group by Estado
              order by QtdTransacoes desc """



In [ ]:
ps.sqldf(query_1, locals())

,Estado,QtdTransacoes
0,SP,21007
1,MG,5845
2,RJ,4281
3,RS,3923
4,SC,3350
5,PR,2598
6,PB,2382
7,DF,2197
8,GO,1150
9,MA,911


In [ ]:
query_2 = """ select
              C.Estado,
              ROUND(COUNT(T.AcumuloID),2) as QtdTransacoes,
              ROUND(SUM(T.ValorTransacao),2) as TotalTransacoes,
              ROUND(AVG(T.ValorTransacao),2) as TicketMedio,
              COUNT(distinct T.IdCliente) as Frequencia,
              strftime('%m', T.DataTransacao) AS Mes,
              strftime('%Y', T.DataTransacao) AS Ano

              from transacoes T
              LEFT join clientes C on T.IdCliente = C.IdCliente
              GROUP BY Ano, Mes
              ORDER BY Ano, Mes"""

In [ ]:
ps.sqldf(query_2, locals())

,Estado,QtdTransacoes,TotalTransacoes,TicketMedio,Frequencia,Mes,Ano
0,RJ,12917.0,1475933.57,114.26,2547,03,2024
1,PR,13945.0,1551098.04,111.23,2602,04,2024
2,SP,13603.0,1620817.82,119.15,2628,05,2024
3,MG,12907.0,1425234.75,110.42,2546,06,2024


In [ ]:
query_3 = """ SELECT
              C.IdCliente,
              SUM(Pontos) as PontosAcumulados,
              CASE
              WHEN SUM(Pontos) BETWEEN 1 AND 100 THEN "1 a 100"
              WHEN SUM(Pontos) BETWEEN 101 AND 300 THEN "101 a 300"
              WHEN SUM(Pontos) BETWEEN 301 AND 500 THEN "301 a 500"
              WHEN SUM(Pontos) BETWEEN 501 AND 1000 THEN "501 a 1000"
              ELSE 'Maior que 1000'
              END AS FaixaPontos
              from transacoes T
              LEFT join clientes C on T.IdCliente = C.IdCliente
              GROUP BY C.IdCliente"""

In [ ]:
ps.sqldf(query_3, locals())

,IdCliente,PontosAcumulados,FaixaPontos
0,410898,2585.0,Maior que 1000
1,416326,1944.0,Maior que 1000
2,417909,6661.0,Maior que 1000
3,421792,1828.0,Maior que 1000
4,452665,1191.0,Maior que 1000
...,...,...,...
2986,42237912,2785.0,Maior que 1000
2987,42241378,2000.0,Maior que 1000
2988,42243438,2240.0,Maior que 1000
2989,42252629,2900.0,Maior que 1000


In [ ]:
query_4 = """ SELECT
              C.IdCliente,
             CAST(julianday('now') - julianday(MAX(T.DataTransacao)) AS INTEGER) AS dias_sem_transacionar
              from transacoes T
              LEFT join clientes C on T.IdCliente = C.IdCliente
              GROUP BY C.IdCliente
              ORDER BY dias_sem_transacionar DESC"""

In [ ]:
ps.sqldf(query_4, locals())


,IdCliente,dias_sem_transacionar
0,39299461,134
1,7688433,134
2,42070513,133
3,33210801,132
4,39100480,131
...,...,...
2986,746741,18
2987,646240,18
2988,470350,18
2989,457589,18


In [ ]:
query_5 = """ SELECT
              C.Estado,
              COUNT(DISTINCT T.IdCliente) AS num_clientes

              from transacoes T
              LEFT join clientes C on T.IdCliente = C.IdCliente
              WHERE ValorTransacao > 100
              GROUP BY C.Estado
              ORDER BY num_clientes DESC
              """

In [ ]:
ps.sqldf(query_5, locals())

,Estado,num_clientes
0,SP,1052
1,MG,291
2,RS,233
3,SC,180
4,PR,159
5,RJ,146
6,DF,116
7,PB,66
8,GO,50
9,MA,43
